# Quantum helper file
This Jupyter notebook contains various helper functions to allow ease of use and to update as code changes progress and evolve. Rather than changing code across all chapters, this will serve as a single source of reference for all functionality.

This Jupyter notebook has been updated to work with the upgraded IBM Quantum® Platform, Python 3.13, Qiskit 2.1, Qiskit Runtime 0.40, and Qiskit Aer 0.17.
|||
|-|-|
|**Author:** |Robert Loredo|
|**Reviewer:** |Alain Chancé|
|**Date:** |August 28, 2025|
|**Version:** |**2.00**<br/>*Details see at the end of this notebook*|
|**References:**|
[Robert Loredo, Learn Quantum Computing with Python and IBM Quantum: Write your own practical quantum programs with Python, Second Edition, Packt, Feb 25, 2025](https://www.packtpub.com/en-us/product/learn-quantum-computing-with-python-and-ibm-quantum-9781803240923)
[Learning-Quantum-Computing-with-Python-and-IBM-Quantum-Second-Edition/helper_file_1.0.ipynb](https://github.com/PacktPublishing/Learning-Quantum-Computing-with-Python-and-IBM-Quantum-Second-Edition/blob/main/helper_file_1.0.ipynb)
[Packt's code repository for the book](https://github.com/PacktPublishing/Learning-Quantum-Computing-with-Python-and-IBM-Quantum-Second-Edition)
[LICENSE](https://github.com/PacktPublishing/Learning-Quantum-Computing-with-Python-and-IBM-Quantum-Second-Edition/blob/main/LICENSE)
[StatevectorSampler](https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.StatevectorSampler)
[Exact simulation with Qiskit SDK primitives - Get the probability distribution or measurement outcome](https://docs.quantum.ibm.com/guides/simulate-with-qiskit-sdk-primitives#get-the-probability-distribution-or-measurement-outcome)
[StatevectorSampler #2219](https://github.com/Qiskit/qiskit-aer/issues/2219)
<br/>

In [1]:
# Load helper file
%run setup_save_account.ipynb


In [2]:
# Import libraries and objects
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_ibm_runtime import Options
from qiskit_ibm_runtime import Session
from qiskit.primitives import StatevectorSampler
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.quantum_info import Statevector
from qiskit_ibm_runtime.fake_provider import FakeManilaV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import *

# Added by Alain Chancé
import qiskit.quantum_info as qi


service = QiskitRuntimeService()

In [3]:
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.providers.fake_provider import GenericBackendV2

def create_circuit(num_qubits, add_measurements):

    # Create a circuit with classical control
    meas = ClassicalRegister(num_qubits)
    qc = QuantumCircuit(num_qubits)
    qc.add_register(meas)
    if add_measurements:
        for i in range(num_qubits):
            qc.measure(i, meas[i])
 
    # Define backend with custom basis gates
    #backend = GenericBackendV2(
     #   num_qubits=num_qubits,
     #   basis_gates=["ecr", "id", "rz", "sx", "x"],
     #   control_flow=True,
    #)
 
    return qc

In [4]:
# Run a Statevector simulator: 
def run_sv_simulator(qc):
    from qiskit.quantum_info import Statevector
    statevector = Statevector(qc)
    return statevector 



In [5]:
# Run a Unitary simulator: 
def run_unitary_simulator(qc):
    from qiskit.quantum_info import Operator
    result = Operator(qc).data
    return result


In [6]:
# Run a QASM simulator: 
def run_qasm_simulator(qc):
    from qiskit.providers.basic_provider import BasicSimulator
    backend = BasicSimulator()  
    result = backend.run(qc).result()
    return result


In [7]:
# Execute circuit on the the State vector sampler
def run_sv_circuit(qc):
    # Run the circuit and return the state vector object result
    stateVectorResult = Statevector(qc)
    #qsphere = stateVectorResult.draw('qsphere')
    #bloch_sphere = stateVectorResult.draw('bloch')
    #circuit_image = qc.draw(output='mpl')
    return stateVectorResult
    

In [8]:
# Execute circuit on the Basic Aer qasm simulator, RENAMED from run_qasm_circuit to run_simulated_circuit
# Modified by Alain Chancé
#def run_simulated_circuit(qc, backend, options):
def run_simulated_circuit(qc, backend):
    #Check if option object is set
    #if options == None:
        #options = Options(optimization_level=3)
    
    #Check if backend is defined
    if backend == None:
        #backend = BasicSimulator()
        backend = AerSimulator(method='statevector')
    
    transpiled_qc = transpile(qc, backend)
    result = backend.run(transpiled_qc).result()

    stateVectorResult = qi.Statevector.from_instruction(transpiled_qc)
    
    return transpiled_qc, result, stateVectorResult

In [9]:
# Execute circuit on the Basic Aer qasm simulator
def simulate_on_sampler(qc, backend, options):

    # Construct an ideal simulator with Sampler 
    from qiskit.primitives import StatevectorSampler
    sampler = StatevectorSampler()

    # Transpile circuit
    pm = generate_preset_pass_manager(optimization_level=1)
    transpiled_qc = pm.run(qc)

    # Run using sampler
    state_vector_result = sampler.run([qc])
    
    result = state_vector_result.result()
    
    return transpiled_qc, result, state_vector_result

In [10]:
# Execute a circuit on a real backend - Sampler
def execute_circuit(qc, simulator, service, backend, options):
    # If executing on a qasm simulator
    if simulator:
        # Modified by Alain Chancé
        #return run_qasm_circuit(qc, backend, options)
        return run_qasm_simulator(qc)
    
    if options == None:
        options = Options(optimization_level=3)
    
    with Session(service=service, backend=backend) as session:
        # Submit a request to the Sampler primitive within the session.
        sampler = Sampler(session=session, options=options)
        job = sampler.run(circuits=qc)
        result = job.result()
        return result

In [11]:
# Will run the circuit on the state vector (sv) simulator
# Returns state vector results, circuit diagram, Sphere & Bloch sphere 
def execute_circuit_sv(quantum_circuit):
    stateVectorResults = run_sv_circuit(quantum_circuit)
    
    #Draw the circuit diagram
    circuit_diagram = quantum_circuit.draw(output="mpl")
    #Draw the Qsphere 
    q_sphere = stateVectorResults.draw('qsphere')
    #Draw the Bloch sphere 
    bloch_sphere = stateVectorResults.draw('bloch')   
    #Return the results, circuit diagram, and QSphere		
    return stateVectorResults, circuit_diagram, q_sphere, bloch_sphere

In [12]:
# Will execute the circuit on the qasm simulator
# Returns results, circuit diagram, and histogram 
def execute_circuit_returns(quantum_circuit):
    # Modified by Alain Chancé
    # StatevectorSampler, https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.StatevectorSampler
    
    #from qiskit.primitives import Sampler
    from qiskit.primitives import StatevectorSampler
    
    #sampler = Sampler()
    # Construct an ideal simulator with SamplerV2
    sampler = StatevectorSampler()
 
    #result = sampler.run(quantum_circuit, shots=1024).result()
    result = sampler.run([quantum_circuit], shots=1024).result()
    
    #quasi_dists = result.quasi_dists

    # Convert the output to bit strings
    #counts = quasi_dists[0].binary_probabilities()
    #counts = result.get_counts(quantum_circuit)

    # Exact simulation with Qiskit SDK primitives
    # https://docs.quantum.ibm.com/guides/simulate-with-qiskit-sdk-primitives#get-the-probability-distribution-or-measurement-outcome
    
    # Access result data for PUB 0
    data_pub = result[0].data
    
    # Get counts for the classical register "meas"
    # measure_all must be used with data.meas
    # measure_active must be used with data.measure
    # See Unexpected Error Thrown by StatevectorSampler #2219 - https://github.com/Qiskit/qiskit-aer/issues/2219
    
    counts = data_pub.meas.get_counts()
    #print(f"The counts are: {counts}")

    #Draw the circuit diagram
    circuit_diagram = quantum_circuit.draw(output="mpl")
    
    #Create a histogram of the counts
    histogram = plot_distribution(counts)
    
    #Return the results, circuit diagram, and histogram
    return counts, circuit_diagram, histogram

In [13]:
# Will execute the circuit on the qasm simulator
# Returns results, circuit diagram, and histogram 
def execute_param_circuit(quantum_circuit, params):
    # Modified by Alain Chancé
    # StatevectorSampler, https://docs.quantum.ibm.com/api/qiskit/qiskit.primitives.StatevectorSampler
    
    #from qiskit.primitives import Sampler
    from qiskit.primitives import StatevectorSampler
    
    #sampler = Sampler()
    # Construct an ideal simulator with SamplerV2
    sampler = StatevectorSampler()
    
    #result = sampler.run(quantum_circuit, shots=1024).result()
    pub = (quantum_circuit, params)
    job = sampler.run([pub], shots=1024)

    # Exact simulation with Qiskit SDK primitives
    # https://docs.quantum.ibm.com/guides/simulate-with-qiskit-sdk-primitives#get-the-probability-distribution-or-measurement-outcome
    
    # Extract the result for the 0th pub (this example only has one pub).
    result = job.result()[0]
    data_pub = result[0].data
    
    #quasi_dists = result.quasi_dists

    # Convert the output to bit strings
    #counts = quasi_dists[0].binary_probabilities()
    
    # Get counts for the classical register "meas"
    # measure_all must be used with data.meas
    # measure_active must be used with data.measure
    # See Unexpected Error Thrown by StatevectorSampler #2219 - https://github.com/Qiskit/qiskit-aer/issues/2219
    counts = data_pub.meas.get_counts()
    #print(f"The counts are: {counts}")

    # Draw the circuit diagram
    circuit_diagram = quantum_circuit.draw(output="mpl")
    
    # Create a histogram of the counts
    histogram = plot_distribution(counts)
    
    #Return the results, circuit diagram, and histogram
    return counts, circuit_diagram, histogram

In [14]:
#Flag to set to run test cell below, simply change False to True:
test = False
if test:
    import qiskit
    from qiskit.primitives import StatevectorSampler

    # Construct an ideal simulator with SamplerV2 
    sampler = StatevectorSampler()

    # Generate 3-qubit GHZ state
    circuit = qiskit.QuantumCircuit(3)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.cx(1, 2)
    circuit.measure_all()

    # Transpile circuit
    pm = generate_preset_pass_manager(optimization_level=1)
    isa_circuit = pm.run(circuit)

    # Run using sampler
    result = sampler.run([circuit]).result()

    # Access result data for PUB 0
    data_pub = result[0].data

    # Access bitstring for the classical register "meas"
    bitstrings = data_pub.meas.get_bitstrings()
    print(f"The number of bitstrings is: {len(bitstrings)}")

    # Get counts for the classical register "meas"
    counts = data_pub.meas.get_counts()
    print(f"The counts are: {counts}")

## Qiskit packages versions

In [15]:
import qiskit
import qiskit_ibm_runtime
import qiskit_aer

print(f'Qiskit: {qiskit.__version__}')
print(f'Qiskit IBM Runtime: {qiskit_ibm_runtime.__version__}')
print(f"Qiskit Aer version: {qiskit_aer.__version__}")

Qiskit: 2.1.1
Qiskit IBM Runtime: 0.40.1
Qiskit Aer version: 0.17.1


## Show Python version

In [16]:
%%bash
which python
python --version

/home/alain/miniconda3/bin/python
Python 3.13.5
